# Undirected GraphSAGE on the CORA citation network using StellarGraph and Neo4j

In [1]:
import pandas as pd
import os

import stellargraph as sg
from stellargraph.connector.neo4j import Neo4JGraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

import numpy as np
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection

import time

### Loading the CORA data from Neo4J

It is assumed that the cora dataset has already been loaded into Neo4J. This notebook demonstrates how to load cora dataset into Neo4J [here](./load-cora-into-neo4j.ipynb).

It is still required to load the node features into memory. We use ```py2neo```, which provides tools to connect to Neo4J databases from Python applications. ```py2neo``` documentation could be found [here](https://py2neo.org/v4/).

In [2]:
import py2neo

# connect to the graph database
neo4j_graphdb = py2neo.Graph("http://localhost:7474", username="neo4j", password="cora")

In [3]:
def get_node_data_from_neo4j(neo4j_graphdb):
    fetch_node_query = "MATCH (node) RETURN id(node), properties(node)"
    # run the query
    node_records = neo4j_graphdb.run(fetch_node_query)
    # convert the node records into pandas dataframe
    return pd.DataFrame(node_records).rename(columns={0: "id", 1: "attr"})


start = time.time()
node_data = get_node_data_from_neo4j(neo4j_graphdb)
end = time.time()

print(f"{end - start:.2f} s: Loaded node data from neo4j database to memory")

5.09 s: Loaded node data from neo4j database to memory


Extract the node_features

In [4]:
node_data = node_data.set_index("id")
attribute_df = node_data["attr"].apply(pd.Series)
node_data = attribute_df.drop(labels=["ID"], axis=1)
node_features = pd.DataFrame(node_data["features"].values.tolist(), index=node_data.index)
node_data.head(5)

,features,subject
id,,
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Neural_Networks
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",Rule_Learning
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reinforcement_Learning
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reinforcement_Learning
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Probabilistic_Methods


We aim to train a graph-ML model that will predict the "subject" attribute on the nodes. These subjects are one of 7 categories:

In [5]:
labels = np.array(node_data["subject"])
set(labels)

{'Case_Based',
 'Genetic_Algorithms',
 'Neural_Networks',
 'Probabilistic_Methods',
 'Reinforcement_Learning',
 'Rule_Learning',
 'Theory'}

### Splitting the data

For machine learning we want to take a subset of the nodes for training, and use the rest for testing. We'll use scikit-learn again to do this

In [6]:
train_data, test_data = model_selection.train_test_split(
    node_data, train_size=0.1, test_size=None, stratify=node_data["subject"]
)

### Converting to numeric arrays

For our categorical target, we will use one-hot vectors that will be fed into a soft-max Keras layer during training.

In [7]:
target_encoding = feature_extraction.DictVectorizer(sparse=False)

train_targets = target_encoding.fit_transform(train_data[["subject"]].to_dict("records"))
test_targets = target_encoding.transform(test_data[["subject"]].to_dict("records"))

## Creating the GraphSAGE model in Keras

Now create a StellarGraph object storing node data. Since the subgraph is sampled directly from Neo4J, we only need to retain the node features and do not need to store any edges.

In [8]:
G = sg.StellarGraph(nodes={"paper": node_features}, edges={})

In [9]:
print(G.info())

<class 'stellargraph.core.graph.StellarGraph'>: Undirected multigraph
 Nodes: 2708, Edges: 0

 Node types:
  paper: [2708]
    Edge types: 

 Edge types:


To feed data from the graph to the Keras model we need a data generator that feeds data from the graph to the model. The generators are specialized to the model and the learning task so we choose the `Neo4JGraphSAGENodeGenerator` as we are predicting node attributes with a GraphSAGE model, sampling directly from Neo4J database.

We need two other parameters, the `batch_size` to use for training and the number of nodes to sample at each level of the model. Here we choose a two-level model with 10 nodes sampled in the first layer, and 5 in the second.

In [10]:
batch_size = 50
num_samples = [10, 5]

A `Neo4JGraphSAGENodeGenerator` object is required to send the node features in sampled subgraphs to Keras.

In [11]:
generator = Neo4JGraphSAGENodeGenerator(G, batch_size, num_samples, neo4j_graphdb)

Using the `generator.flow()` method, we can create iterators over nodes that should be used to train, validate, or evaluate the model. For training we use only the training nodes returned from our splitter and the target values. The `shuffle=True` argument is given to the `flow` method to improve training.

In [12]:
train_gen = generator.flow(train_data.index, train_targets, shuffle=True)

Now we can specify our machine learning model, we need a few more parameters for this:

 * the `layer_sizes` is a list of hidden feature sizes of each layer in the model. In this example we use 32-dimensional hidden node features at each layer.
 * The `bias` and `dropout` are internal parameters of the model. 

In [13]:
graphsage_model = GraphSAGE(
    layer_sizes=[32, 32], generator=generator, bias=True, dropout=0.5,
)

Now we create a model to predict the 7 categories using Keras softmax layers.

In [14]:
x_inp, x_out = graphsage_model.build()
prediction = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)

### Training the model

Now let's create the actual Keras model with the graph inputs `x_inp` provided by the `graph_model` and outputs being the predictions from the softmax layer

In [15]:
model = Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=optimizers.Adam(lr=0.005),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

Train the model, keeping track of its loss and accuracy on the training set, and its generalisation performance on the test set (we need to create another generator over the test data for this)

In [16]:
test_gen = generator.flow(test_data.index, test_targets)

In [17]:
history = model.fit_generator(
    train_gen, epochs=20, validation_data=test_gen, verbose=2, shuffle=False
)

Epoch 1/20
6/6 - 4s - loss: 1.8717 - acc: 0.2889 - val_loss: 1.7331 - val_acc: 0.4212
Epoch 2/20
6/6 - 3s - loss: 1.6507 - acc: 0.4556 - val_loss: 1.5808 - val_acc: 0.4959
Epoch 3/20
6/6 - 3s - loss: 1.5011 - acc: 0.6185 - val_loss: 1.4209 - val_acc: 0.6530
Epoch 4/20
6/6 - 4s - loss: 1.3208 - acc: 0.8148 - val_loss: 1.2929 - val_acc: 0.7256
Epoch 5/20
6/6 - 3s - loss: 1.2038 - acc: 0.8630 - val_loss: 1.1804 - val_acc: 0.7699
Epoch 6/20
6/6 - 3s - loss: 1.0483 - acc: 0.9185 - val_loss: 1.0920 - val_acc: 0.7838
Epoch 7/20
6/6 - 4s - loss: 0.9495 - acc: 0.9259 - val_loss: 1.0150 - val_acc: 0.7994
Epoch 8/20
6/6 - 3s - loss: 0.8638 - acc: 0.9407 - val_loss: 0.9414 - val_acc: 0.8167
Epoch 9/20
6/6 - 3s - loss: 0.7612 - acc: 0.9593 - val_loss: 0.8881 - val_acc: 0.8191
Epoch 10/20
6/6 - 3s - loss: 0.6842 - acc: 0.9704 - val_loss: 0.8424 - val_acc: 0.8240
Epoch 11/20
6/6 - 3s - loss: 0.6317 - acc: 0.9630 - val_loss: 0.8045 - val_acc: 0.8105
Epoch 12/20
6/6 - 3s - loss: 0.5641 - acc: 0.9667 - 

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_history(history):
    metrics = sorted(history.history.keys())
    metrics = metrics[:len(metrics)//2]
    for m in metrics:
        # summarize history for metric m
        plt.plot(history.history[m])
        plt.plot(history.history['val_' + m])
        plt.title(m)
        plt.ylabel(m)
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='best')
        plt.show()

In [19]:
plot_history(history)

Now we have trained the model we can evaluate on the test set.

In [20]:
test_metrics = model.evaluate_generator(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

### Making predictions with the model

Now let's get the predictions themselves for all nodes using another node iterator:

In [21]:
all_nodes = node_data.index
all_mapper = generator.flow(all_nodes)
all_predictions = model.predict_generator(all_mapper)

These predictions will be the output of the softmax layer, so to get final categories we'll use the `inverse_transform` method of our target attribute specifcation to turn these values back to the original categories

In [22]:
node_predictions = target_encoding.inverse_transform(all_predictions)

Let's have a look at a few:

In [23]:
results = pd.DataFrame(node_predictions, index=all_nodes).idxmax(axis=1)
df = pd.DataFrame({"Predicted": results, "True": node_data["subject"]})
df.head(10)

Please refer to [directed-graphsage-on-cora-example.ipynb](./../../node-classification/graphsage/graphsage-cora-node-classification-example.ipynb) for **node embedding visualization**.